In [1]:
#!{sys.executable} -m pip install Pillow

from pathlib import Path
from PIL import Image
import easyocr
import numpy as np
from sklearn.cluster import DBSCAN
import numpy as np
import os
from pathlib import Path
from paddleocr import PaddleOCR
from PIL import Image, ImageDraw, ImageFont



In [10]:
# --------------------------
# CONFIG
# --------------------------
input_folder = Path(r"input")
output_folder = Path("output")
output_folder.mkdir(exist_ok=True)

allow_chars = "ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789!?.,-…'\""
replacements = {"PottyWords": "&$%&#$%"}  # Dictionary of word replacements
font_path = "fonts/CCMeanwhileBold.ttf"  # Use a comic font if you have one
font_size = 15  # Adjust size to fit speech bubbles

# --------------------------
# LOAD IMAGES
# --------------------------
image_files = sorted(input_folder.rglob("*.webp"))
images = [Image.open(f) for f in image_files]

# --------------------------
# INITIALIZE OCR
# --------------------------
reader = easyocr.Reader(['en'], gpu=False)
ocr_results_all = []

# --------------------------
# OCR ALL IMAGES
# --------------------------
for img in images:
    results = reader.readtext(np.array(img), allowlist=allow_chars, width_ths=0.05)
    ocr_results_all.append(results)

# --------------------------
# REPLACE WORDS
# --------------------------
def replace_words(results, replacements):
    """
    Replace words in OCR results according to a dictionary.
    """
    new_results = []
    for bbox, text, conf in results:
        new_text = " ".join([replacements.get(word.upper(), word) for word in text.split()])
        new_results.append((bbox, new_text, conf))
    return new_results

ocr_results_modified = [replace_words(r, replacements) for r in ocr_results_all]

# --------------------------
# DRAW TEXT WITH PRE-DRAWN BACKGROUNDS
# --------------------------
def draw_text_on_image_no_overlap(img, results, font_path=font_path, font_size=font_size, padding=25):
    """
    Draw white background rectangles first for all words, then draw all text on top.
    Compatible with Pillow >= 10.
    """
    pil_img = img.copy()
    draw = ImageDraw.Draw(pil_img)
    font = ImageFont.truetype(font_path, font_size)

    # Step 1: Draw all white backgrounds
    for bbox, text, conf in results:
        x_center = int(np.mean([p[0] for p in bbox]))
        y_center = int(np.mean([p[1] for p in bbox]))
        
        # Use textbbox to get width and height
        bbox_text = draw.textbbox((0, 0), text, font=font)
        text_width = bbox_text[2] - bbox_text[0]
        text_height = bbox_text[3] - bbox_text[1]

        x0 = x_center - text_width // 2 - padding
        y0 = y_center - text_height // 2 - padding
        x1 = x_center + text_width // 2 + padding
        y1 = y_center + text_height // 2 + padding
        
        draw.rectangle([x0, y0, x1, y1], fill="white")

    # Step 2: Draw all text on top
    for bbox, text, conf in results:
        x_center = int(np.mean([p[0] for p in bbox]))
        y_center = int(np.mean([p[1] for p in bbox]))
        draw.text((x_center, y_center), text, fill="black", font=font, anchor="mm")

    return pil_img

# --------------------------
# PROCESS AND SAVE IMAGES
# --------------------------
for idx, img in enumerate(images):
    modified_img = draw_text_on_image_no_overlap(img, ocr_results_modified[idx])
    output_path = output_folder / f"page_{idx+1:03d}.png"
    modified_img.save(output_path)
    print(f"Saved modified page with bubbles: {output_path}")

Using CPU. Note: This module is much faster with a GPU.


Saved modified page with bubbles: output\page_001.png
Saved modified page with bubbles: output\page_002.png
Saved modified page with bubbles: output\page_003.png
Saved modified page with bubbles: output\page_004.png
Saved modified page with bubbles: output\page_005.png
Saved modified page with bubbles: output\page_006.png
Saved modified page with bubbles: output\page_007.png
Saved modified page with bubbles: output\page_008.png
Saved modified page with bubbles: output\page_009.png


In [13]:
from pathlib import Path
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import easyocr
import re

# --------------------------
# CONFIG
# --------------------------
input_folder = Path(r"input")
output_folder = Path("output")
output_folder.mkdir(exist_ok=True)

allow_chars = "ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789!?.,-…'\""
replacements = {"PottyWords": "&$%&#$%"}
font_path = "fonts/CCMeanwhileBold.ttf"
font_size = 15

# --------------------------
# INITIALIZE OCR
# --------------------------
reader = easyocr.Reader(['en'], gpu=False)

# --------------------------
# HELPER FUNCTIONS
# --------------------------
def natural_sort_key(path):
    return [int(text) if text.isdigit() else text.lower()
            for text in re.split(r'(\d+)', str(path))]

def replace_words(results, replacements):
    new_results = []
    for bbox, text, conf in results:
        new_text = " ".join([replacements.get(word.upper(), word) for word in text.split()])
        new_results.append((bbox, new_text, conf))
    return new_results

def draw_text_on_image_no_overlap(img, results, font_path, font_size, padding=25):
    pil_img = img.copy()
    draw = ImageDraw.Draw(pil_img)
    font = ImageFont.truetype(font_path, font_size)

    # Step 1: Draw all white backgrounds
    for bbox, text, conf in results:
        x_center = int(np.mean([p[0] for p in bbox]))
        y_center = int(np.mean([p[1] for p in bbox]))
        bbox_text = draw.textbbox((0, 0), text, font=font)
        text_width = bbox_text[2] - bbox_text[0]
        text_height = bbox_text[3] - bbox_text[1]
        x0 = x_center - text_width // 2 - padding
        y0 = y_center - text_height // 2 - padding
        x1 = x_center + text_width // 2 + padding
        y1 = y_center + text_height // 2 + padding
        draw.rectangle([x0, y0, x1, y1], fill="white")

    # Step 2: Draw text
    for bbox, text, conf in results:
        x_center = int(np.mean([p[0] for p in bbox]))
        y_center = int(np.mean([p[1] for p in bbox]))
        draw.text((x_center, y_center), text, fill="black", font=font, anchor="mm")

    return pil_img

# --------------------------
# PROCESS FOLDERS
# --------------------------
# Get all folders (including input_folder itself)
folders = [f for f in input_folder.rglob('*') if f.is_dir()] + [input_folder]

for folder in sorted(set(folders)):
    # Gather images (jpg, jpeg, webp)
    image_files = sorted(
        list(folder.glob("*.webp")) +
        list(folder.glob("*.jpg")) +
        list(folder.glob("*.jpeg")),
        key=natural_sort_key
    )

    if not image_files:
        continue  # skip empty folders

    print(f"\nProcessing folder: {folder}")

    # OCR + replacement
    ocr_results_all = []
    for img_path in image_files:
        img = Image.open(img_path)
        results = reader.readtext(np.array(img), allowlist=allow_chars, width_ths=0.05)
        ocr_results_all.append(replace_words(results, replacements))

    # Output subfolder mirrors input structure
    relative_path = folder.relative_to(input_folder)
    output_subfolder = output_folder / relative_path
    output_subfolder.mkdir(parents=True, exist_ok=True)

    # Save processed images
    for idx, img_path in enumerate(image_files):
        img = Image.open(img_path)
        modified_img = draw_text_on_image_no_overlap(img, ocr_results_all[idx], font_path, font_size)
        output_path = output_subfolder / f"{img_path.stem}.png"
        modified_img.save(output_path)
        print(f"  Saved: {output_path}")


Using CPU. Note: This module is much faster with a GPU.



Processing folder: input\Volume_1_Chapter_1_That's_How_Love_Starts,_Ya_Know!
  Saved: output\Volume_1_Chapter_1_That's_How_Love_Starts,_Ya_Know!\page_1.png
  Saved: output\Volume_1_Chapter_1_That's_How_Love_Starts,_Ya_Know!\page_2.png
  Saved: output\Volume_1_Chapter_1_That's_How_Love_Starts,_Ya_Know!\page_3.png
  Saved: output\Volume_1_Chapter_1_That's_How_Love_Starts,_Ya_Know!\page_4.png
  Saved: output\Volume_1_Chapter_1_That's_How_Love_Starts,_Ya_Know!\page_5.png
  Saved: output\Volume_1_Chapter_1_That's_How_Love_Starts,_Ya_Know!\page_6.png
  Saved: output\Volume_1_Chapter_1_That's_How_Love_Starts,_Ya_Know!\page_7.png
  Saved: output\Volume_1_Chapter_1_That's_How_Love_Starts,_Ya_Know!\page_8.png
  Saved: output\Volume_1_Chapter_1_That's_How_Love_Starts,_Ya_Know!\page_9.png
  Saved: output\Volume_1_Chapter_1_That's_How_Love_Starts,_Ya_Know!\page_10.png

Processing folder: input\Volume_1_Chapter_2_That's_A_Space_Alien,_Ain't_It!
  Saved: output\Volume_1_Chapter_2_That's_A_Space_Alie